In [1]:
import pandas as pd
from libraries.utility import Utility as mutil
from libraries.db_base import db_base
import os
import time

from libraries.utility import Utility as mutil
from libraries.db_base import db_base
from libraries.db_ins_fake_data import fake_data_to_db
from libraries.custom_excel_output import custom_excel_output
import pandas as pd
import pandasql as psql
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import warnings
import time
import os
import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))


from pyspark.sql import SparkSession



In [2]:

u = mutil()

bq_credential_file = os.path.join(u.get_this_dir(),"dbs","servers","bigquery","bigquery1","tokens","brave-sonar-367918-74b1d5b6db90.json")

    #Location of database drivers
    #postgres
    #/usr/share/java/postgresql.jar
    #mysql
    #/usr/share/java/mysql-connector-j-8.2.0.jar
    #mssql
    #/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar

def create_spark_sesssion():
    # Create SparkSession
    spark = SparkSession.builder \
        .appName("MasterSession") \
        .config("spark.jars", "/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar") \
        .config("spark.jars", "/usr/share/java/postgresql.jar") \
        .config("spark.jars", "/usr/share/java/mysql-connector-j-8.2.0.jar") \
        .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.21.1') \
        .config('spark.jars', 'https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
        .config("credentialsFile", bq_credential_file) \
        .config("parentProject", "brave-sonar-367918") \
        .getOrCreate()
    return spark

def test_sql_server_connection(query,spark):
    

    # Start the timer
    start_time = time.time()
    


    # Database connection properties
    database_url = "jdbc:sqlserver://mssql1:1433;databaseName=products"
    database_properties = {
        "user": "sa",
        "password": "Python2028",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }

    # Read data from MSSQL
    #df = spark.read.jdbc(url=database_url, table="products", properties=database_properties)

    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)

    # Show the data
    #df.show()
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    return df


def test_postgress_sql(query,spark):
  

    # Start the timer
    start_time = time.time()
    

    # Database connection properties
    database_url = "jdbc:postgresql://postsql1:5432/finance"
    database_properties = {
        "user": "postgres",
        "password": "Python2028",
        "driver": "org.postgresql.Driver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }
    
    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)

    # Show the data
    #df.show()
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    return df

def test_mysql_server(query,spark):
    
    # Start the timer
    start_time = time.time()
    

    # Database connection properties
    database_url = "jdbc:mysql://mysql1:3306/customers"
    database_properties = {
        "user": "root",
        "password": "Python2028",
        "driver": "com.mysql.cj.jdbc.Driver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }
    
    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    
    return df
    


In [3]:

u = mutil()
# Start the timer
start_time = time.time()

# Initialize a SparkSession
spark = create_spark_sesssion()


dbprod = db_base("products")
get_product_info_sql=dbprod.get_sql_query_from_query_key("get_product_info")

query= "(" + get_product_info_sql + ") AS query_table"

prod_info=test_sql_server_connection(query=query,spark=spark)

dbfin = db_base("finance",svr_type='postsql')

get_account_activity = dbfin.get_sql_query_from_query_key("get_account_activity")

query= "(" + get_account_activity + ") AS query_table"

fin_account_activity=test_postgress_sql(query,spark)

dbcust = db_base("customers",svr_type='mysql')

query = dbcust.get_sql_query_from_query_key("get_customer_product_history1")

query = "(" + query + ") AS query_table"

cust_products = test_mysql_server(query,spark)


/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9c59f428-d4be-4e14-bec4-f1b31b951b0d;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.21.1 in central
:: resolution report :: resolve 89ms :: artifacts dl 3ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.21.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retriev

('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found (0) (SQLDriverConnect)")
Database : products, Connection Good: False
Time elapsed: 0 minutes and 1 seconds
connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

Database : finance, Connection Good: False
Time elapsed: 0 minutes and 0 seconds
2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)
Database : customers, Connection Good: False
Time elapsed: 0 minutes and 0 seconds


In [8]:

# Register DataFrames as temporary views
cust_products.createOrReplaceTempView("cust_products")
prod_info.createOrReplaceTempView("prod_info")
fin_account_activity.createOrReplaceTempView("fin_account_activity")


# SQL query
sql_query = '''
SELECT
    DISTINCT
    cp.id AS customer_id,
    cp.f_name,
    cp.l_name,
    p.product_name,
    p.product_type,
    fa.amt_usd,
    fa.post_date,
    fa.channel_desc,
    fa.partner_desc,
    fa.location_name,
    fa.account_name,
    fa.account_type
FROM cust_products cp
JOIN prod_info p ON p.id = cp.product_id
JOIN fin_account_activity fa ON fa.customer_id = cp.id AND fa.product_id = cp.product_id
'''

# Execute SQL query
result_rdd = spark.sql(sql_query)
result_df = result_rdd.toPandas()
#result_df_bq = result_rdd.toDF()

#filename = os.path.join(u.get_this_dir(),"outfile.csv")

#result_df.to_csv(filename, index=False, header=True)


In [5]:

#Write the stuff to my bigquery database.
result_rdd.write \
  .format("bigquery") \
  .option("project", "brave-sonar-367918") \
  .option("table", "site_traffic.financial_data") \
  .option("writeMethod", "direct") \
  .mode("append") \
  .save()


In [6]:
import boto3 
import psycopg2

#AWS Redshift Credentials
import json
import os
from io import StringIO

#Path to the uploaded file using os.path.join
#dbs/servers/redshift/test_data/tokens
file_path = os.path.join(u.get_this_dir(),"dbs","servers","redshift","test_data","tokens","redshift_iam.json")

# Opening the JSON file
with open(file_path, 'r') as file:
    credentials = json.load(file)

# Storing the credentials in variables
AWS_redshift_access_key = credentials.get('access_key')
AWS_redshift_secret_access_key = credentials.get('secret_access_key')
AWS_redshift_database = credentials.get('redshift_database')
AWS_redshift_admin_user = credentials.get('redshift_admin_user')
AWS_redshift_admin_pw = credentials.get('redshift_admin_pw')
bucket_name = credentials.get('bucket_name')

#S3 Bucket Details
bucket_name = bucket_name
object_name = 'your-object-name.csv'

# Save DataFrame to an in-memory buffer
csv_buffer = StringIO()
result_df.to_csv(csv_buffer, index=False)

# Create S3 client
s3_client = boto3.client('s3', aws_access_key_id=AWS_redshift_access_key, aws_secret_access_key=AWS_redshift_secret_access_key)

# Upload the DataFrame to S3
s3_client.put_object(Bucket=bucket_name, Body=csv_buffer.getvalue(), Key=object_name)



{'ResponseMetadata': {'RequestId': 'KJ7J704R9GHR4ACE',
  'HostId': 'gwooRZdppuaJ9hiDs6eOkVMU177s3BLOZilqgxJE8fshzUHEPYOOlqnyuxUgGGr7fOfQxf5Wn1g=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'gwooRZdppuaJ9hiDs6eOkVMU177s3BLOZilqgxJE8fshzUHEPYOOlqnyuxUgGGr7fOfQxf5Wn1g=',
   'x-amz-request-id': 'KJ7J704R9GHR4ACE',
   'date': 'Wed, 03 Jan 2024 05:47:00 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f75d14f5ccc8b98161dc7627b87e6787"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"f75d14f5ccc8b98161dc7627b87e6787"',
 'ServerSideEncryption': 'AES256'}

In [7]:

import psycopg2

# JDBC URL details
jdbc_url = "jdbc:redshift://shift4-example.789668808708.us-west-2.redshift-serverless.amazonaws.com:5439/dev"
split_url = jdbc_url.split("//")[1].split(":")
redshift_endpoint = split_url[0]
redshift_port = split_url[1].split("/")[0]
AWS_redshift_database = split_url[1].split("/")[1]

# Establish a connection to Redshift
conn = psycopg2.connect(
    dbname=AWS_redshift_database,
    user=AWS_redshift_admin_user,
    password=AWS_redshift_admin_pw,
    host=redshift_endpoint,
    port=redshift_port
)
cursor = conn.cursor()

create_table_sql = f"""
CREATE TABLE IF NOT EXISTS financial_data
(
  customer_id BIGINT,
  f_name VARCHAR(255),
  l_name VARCHAR(255),
  product_name VARCHAR(255),
  product_type VARCHAR(255),
  amt_usd VARCHAR(255),
  post_date TIMESTAMP,
  channel_desc VARCHAR(255),
  partner_desc VARCHAR(255),
  location_name VARCHAR(255),
  account_name VARCHAR(255),
  account_type VARCHAR(255)
);
"""

# Execute the create table command
cursor.execute(create_table_sql)

s3_path = f"s3://{bucket_name}/{object_name}"

# SQL COPY command to load data from S3 to Redshift
copy_cmd = f"""
COPY {table_name}
FROM '{s3_path}'
ACCESS_KEY_ID '{AWS_redshift_access_key}'
SECRET_ACCESS_KEY '{AWS_redshift_secret_access_key}'
CSV;  -- Assuming the file is in CSV format, modify if necessary
"""


# Execute the COPY command
cursor.execute(copy_cmd)
conn.commit()

# Close the connection
cursor.close()
conn.close()


OperationalError: connection to server at "shift4-example.789668808708.us-west-2.redshift-serverless.amazonaws.com" (172.31.2.17), port 5439 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:


# Stop the SparkSession when done
spark.stop()


# Stop the timer and calculate elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

# Convert elapsed time to minutes and seconds
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"Overall Time: {minutes} minutes and {seconds} seconds")


In [ ]:
# Show the result
result_df.head()

In [ ]:
dbcust = custom_excel_output(current_database="customers",svr_type='mysql')        

In [ ]:

file_name = os.path.join(dbcust.get_this_dir(),"project_data","tempdir","customer_account_activity.xlsx")

result_df["amt_usd"] = result_df["amt_usd"].replace('[\$,]', '', regex=True).astype(float)

l_dfs = []
l_dfs.append(result_df)
dbcust.write_excel_from_dfs(list_of_dfs=l_dfs,file_name=file_name,add_subtotal_on_top=True)

href_tag = dbcust.get_embedded_href_tag_from_image_file(file_name)
dbcust.nukefile(file_name)
display(HTML(href_tag))
shuffled_df = result_df.sample(frac=1)
display(shuffled_df.head(5).style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]))